In [20]:
# source/s: 
# setup gspread
# https://www.youtube.com/watch?v=bu5wXjz2KvU
#
# setup pydrive
# https://www.youtube.com/watch?v=Z2kfNx3Cgsk&t=121s
#
# https://console.cloud.google.com

# %pip install pydrive
# %pip instll gspread
# %pip install pandas
# %pip install pydrive

import os
import pandas as pd
import numpy as np
import gspread
from datetime import datetime
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [21]:
# setup pydrive
gauth = GoogleAuth()
drive = GoogleDrive(gauth)

folder = "1XawLc3-tpec5EdxcZ6g4qCrSrd4miTeI"

In [22]:
# setup service account
sa = gspread.service_account(filename="sheets_credentials.json")

In [23]:
# list files in backup folder
files = drive.ListFile({'q':f"'{folder}' in parents and trashed=false"}).GetList()

for file in files:
	print(file['title'])
	print(type(file['title']))

20220512-085725.csv
<class 'str'>
20220512-085630.csv
<class 'str'>
20220512-085128.csv
<class 'str'>
20220512-085004.csv
<class 'str'>
20220512-084358.csv
<class 'str'>
20220512-084242.csv
<class 'str'>
20220512-084000.csv
<class 'str'>
20220506-233145.csv
<class 'str'>
20220506-224211.csv
<class 'str'>
20220506-224012.csv
<class 'str'>


In [24]:
# list files in backup folder
files = drive.ListFile({'q':f"'{folder}' in parents and trashed=false"}).GetList()

# get latest filename into a string
# create empty series for list

latest_file_id = None
latest_file_datetime = None
passed_first_trigger = True

# algorithm that keeps the file id that is the latest datetime
for file in files:
	current_file_datetime = datetime.strptime(file['title'][:-4], "%Y%m%d-%H%M%S")
	current_file_id = file['id']

	if passed_first_trigger:
		passed_first_trigger = False
		latest_file_id = current_file_id
		latest_file_datetime = current_file_datetime
		continue
	
	if latest_file_datetime < current_file_datetime:
		latest_file_id = current_file_id
		latest_file_datetime = current_file_datetime

print(latest_file_id)

1ehO21mBuyVCpy3d6QxEDca0GJRNyfL-Y


In [25]:
# download csv locally!!!
# local backup location
backup_dir = os.getcwd() + "\\backup\\"
# open latest backup
latest_backup_file = drive.CreateFile({'id':latest_file_id})

# get datetime into string
datetime_string = latest_file_datetime.strftime("%Y%m%d-%H%M%S") + ".csv"
backup_dl_path = backup_dir + datetime_string

latest_backup_file.GetContentFile(backup_dl_path) # Download content



# TODO: check if this error is the only one that is raised if the backup drive
# is empty ANSWER: it is not, see latest
if latest_file_id == None:
	raise ValueError("No backup file IDs.")

# open into dataframe
# open sv data
# clear sv data
# replace sv data with backup

In [26]:
# open csv locally into dataframe
backup_df = pd.read_csv(backup_dl_path)

# delete backup_dl_file
os.remove(backup_dl_path)

backup_df = backup_df.fillna('')
backup_df.head()

,YEAR,RETAILER,PROPERTY_TYPE,TOTAL_DWELLING,POSTCODE,0L_DAY,RATIO_0,50L_DAY,RATIO_50,EXEMPT_TAX_AREA,HOLIDAY,LOCALITY
0,2020,GWW,residential,25825,3000,331,1.281704,3339,12.92933,tax,False,
1,2020,GWW,residential,2296,3002,11,0.479094,250,10.8885,tax,False,EAST MELBOURNE | E MELBOURNE | JOLIMONT | MELB...
2,2020,GWW,residential,4061,3003,35,0.861857,340,8.372322,tax,False,WEST MELBOURNE | MELBOURNE | MELBOURNE WEST | ...
3,2020,GWW,residential,6506,3008,75,1.152782,711,10.92837,tax,False,DOCKLANDS | WORLD TRADE CENTRE
4,2020,GWW,residential,10221,3011,190,1.858918,454,4.441835,tax,False,FOOTSCRAY | FOOTSCRAY WEST | SEDDON WEST | WES...


In [27]:
# open sv_total
sv_total_data_sh = sa.open("sv_total_data")

# get sheet, clear and update
sv_total_data_ws = sv_total_data_sh.get_worksheet(0)
sv_total_data_ws.clear()
sv_total_data_ws.update([backup_df.columns.values.tolist()] + backup_df.values.tolist())

{'spreadsheetId': '1xktogQXx9ls0t_KxM3UvP81MdmX05Y9Oqp60aaA5fv8',
 'updatedRange': 'Sheet1!A1:L540',
 'updatedRows': 540,
 'updatedColumns': 12,
 'updatedCells': 6480}